In [12]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import os
from plotnine import *
import numpy as np
import sys
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse

In [2]:
#Load original data which from cleaned_data folder on OSF
full_data = pd.read_csv('https://raw.githubusercontent.com/thimaipham/Data/main/morg-2014-emp.csv', 
                        dtype={16: str})

In [4]:
#Check data infomation
full_data.head(2)

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
0,3,2600310997690,January,AL,3151.6801,1692.0,40,43,1,NaN,...,0,0,"Native, Born In US",63,Employment services (5613),630,"Private, For Profit",No,No,Employed-At Work
1,5,75680310997590,January,AL,3457.1138,450.0,40,41,2,NaN,...,2,6,"Native, Born In US",63,Outpatient care centers (6214),5400,"Private, For Profit",No,No,Employed-Absent


In [5]:
# Filter the dataset for the occupation 
# "Lawyers, Judges, magistrates, and other judicial workers" (occupation code 2100)
occupation_code = 2100
data = full_data[full_data['occ2012'] == occupation_code]

# Count the number of observations for this occupation
data.shape[0]

1027

###### This filter data include 1027 observations

### Feture Engineering

In [10]:
# Checking for missing values
missing_values = data.isnull().sum()

# Examining data types
data_types = data.dtypes

# Creating a new column for earnings per hour
data['earnings_per_hour'] = data['earnwke'] / data['uhours']

# Display the results
print(missing_values, data_types) 
print(data[['earnwke', 'uhours', 'earnings_per_hour']].head())


Unnamed: 0             0
hhid                   0
intmonth               0
stfips                 0
weight                 0
earnwke                0
uhours                 0
grade92                0
race                   0
ethnic               981
age                    0
sex                    0
marital                0
ownchild               0
chldpres               0
prcitshp               0
state                  0
ind02                  0
occ2012                0
class                  0
unionmme               0
unioncov              57
lfsr94                 0
earnings_per_hour      0
dtype: int64 Unnamed: 0             int64
hhid                   int64
intmonth              object
stfips                object
weight               float64
earnwke              float64
uhours                 int64
grade92                int64
race                   int64
ethnic               float64
age                    int64
sex                    int64
marital                int64
ownchild  

##### Missing values:
- The `ethnic` column has 981 missing values.
- The `unioncov` column has 57 missing values.
Due to a huge number of missing values in `ethic` column and some missing values in `unioncov` column, I will not choose them in the models of the following steps.  Because we have many other variable options for the models.

##### New column: 
- The column `earnings_per_hour` is created by `earnwke`/`uhours`. This will be the target variable for predictive modeling.

### Dealing with categorical values
- As I observed 'race', 'sex', 'marital', 'grade92' are categorical values. I searches for few days on the Internet on "How to deal with categorical values?" and OneHotEncoding method is recommended to convert categorical variables like 'race', 'sex', 'marital', and 'grade92' into a numerical format. This approach avoids imposing any artificial numerical order on these categories, which could lead to biased results. OneHotEncoding creates separate binary columns for each category, ensuring a more accurate representation in the model.

In [15]:
# Selecting categorical columns for one-hot encoding
categorical_columns = ['race', 'sex', 'marital', 'grade92']

# Initializing OneHotEncoder
encoder = OneHotEncoder(sparse=False, drop='first')

# Fitting and transforming the categorical columns
encoded_categorical = encoder.fit_transform(data[categorical_columns])

# Creating a DataFrame from the encoded categorical data
encoded_categorical_df = pd.DataFrame(encoded_categorical, 
                                      columns=encoder.get_feature_names_out(categorical_columns))

# Dropping original categorical columns from the data
data_transformed = data.drop(categorical_columns, axis=1)

# Merging the original data with the new encoded categorical columns
data_transformed = pd.concat([data_transformed, encoded_categorical_df], axis=1)

# Displaying the first few rows of the transformed data
data_transformed.head().T


,23,295,659,764,901
Unnamed: 0,45.0,656.0,1489.0,1729.0,1996.0
hhid,960021970039060.0,235004820400137.0,97977400095618.0,435002800600122.0,700609983456587.0
intmonth,January,January,January,January,January
stfips,AL,AZ,CA,CA,CA
weight,3811.7857,3434.7076,3166.3863,3060.5871,3057.3628
earnwke,1442.3,1384.61,2307.0,700.0,2884.61
uhours,50.0,40.0,50.0,50.0,50.0
ethnic,NaN,NaN,NaN,NaN,NaN
age,57.0,62.0,47.0,29.0,55.0
ownchild,0.0,0.0,1.0,0.0,0.0
